In [1]:
%pip install openai

Note: you may need to restart the kernel to use updated packages.


In [10]:
from openai import OpenAI
import config

In [11]:
client = OpenAI(
    api_key=config.custom_api_key
)

In [12]:
# Prompt templates
import utils
import base64
import os

In [4]:
def chat_with_gpt(prompt):
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    return completion.choices[0].message.content

In [5]:
# Text example
user_input = "What is the capital of Bosnia and Herzegovina" 
response = chat_with_gpt(user_input)
print(f"Chatbot: {response}")

Chatbot: The capital of Bosnia and Herzegovina is Sarajevo.


In [ ]:
# Image example
image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"

response = client.chat.completions.create(
    model="gpt-4o", 
    messages=[{
        "role": "user",
        "content": [
            {"type": "text", "text": "What's in this image?"},
            {
                "type": "image_url", 
                "image_url": {
                    "url": image_url,
                },
            },
        ],
    }],
)

print(response.choices[0].message.content)

The image depicts a scenic landscape with a wooden boardwalk leading through a lush green field. The sky is blue with scattered clouds, creating a serene and open atmosphere. The surrounding area is filled with tall grass and some bushes, suggesting a natural or park setting.


In [ ]:
system_prompt = """You are a UX expert for web applications.
Your task is to identify usability issues with the
information you get for an application’s view.
An example of a usability issue could be: ‘Lack of
visual feedback on user interactions’.
Respond using app domain language; you must not use
technical terminology or mention code details.
Enumerate the problems identified; add an empty
paragraph after each enumeration; no preceding
or following text."""

app_overview = utils.app_overview
user_task = utils.user_task
source_code = utils.source_code

user_prompt = f"""I have a python web application written in Django about: {app_overview}
The user’s task in this app view is about: {user_task}.
An image of the app view is provided.
Below is the incomplete code for the app view.
This code includes the view’s user interface.
Source Code:
{source_code}"""


In [5]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

def detect_usability_issues(image_filename):
    image_path = os.path.join(os.pardir, "images", image_filename) 
    
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Error: The file '{image_path}' was not found. Check the path.")

    base64_image = encode_image(image_path)

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": user_prompt},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/png;base64,{base64_image}",
                        },
                    },
                ],
            },
        ],
    )

    print(response.choices[0].message.content)

In [16]:
# Example usage
detect_usability_issues("all_courses.png") 

1. **Lack of Visual Feedback for Links and Buttons**: The "View" buttons may not provide enough visual feedback on interaction, such as a hover effect to indicate clickability.

2. **Inconsistent Image Aspect Ratio**: Course images could have inconsistent aspect ratios, affecting the overall alignment and visual consistency.

3. **Text Alignment and Spacing**: Card text description spacing seems inconsistent, which can make reading more difficult.

4. **Responsive Design Issues**: It is unclear if the cards adjust well on different screen sizes due to fixed dimensions within inline styles.

5. **Overuse of Inline Styles**: The extensive use of inline styles can hinder design adjustments and maintenance.

6. **Insufficient Contrast for Text**: The contrast between text and background might not be adequate for readability in some cases, especially for accessibility.

7. **Limited Course Information Display**: The course names and summaries may truncate important information, limiting use

In [11]:
# Addition of the Nielsen's 10 general principles for interaction design

system_prompt = f"""You are a UX expert for web applications. Your task is to identify usability
issues with the information you get for an application’s view. For the identification of usability issues,
you should use the following Nielsen’s usability principles:

NS01—Visibility of system status;
NS02—User control and freedom;
NS03—Help users recognize, diagnose, and recover from from errors;
NS04—Consistency and standards;
NS05—Recognition rather than recall;
NS06—Match Between System and The Real World;
NS07—Error Prevention;
NS08—Flexibility and Efficiency of Use;
NS09—Aesthetic and Minimalist Design;
NS10—Help and Documentation.

A description of the Nielsen's usability principles is given in {utils.nielsen_principles}.

Based on the above information, analyze and evaluate the website’s performance referencing the information you get for an application’s view,
which includes the image of the web app, code, app overview and user task, according to Nielsen’s usability
principles. Rate each area on an integer scale from e to 5, adjusting the score up or down
from a baseline of 3 points based on significant differences. Finally, provide justifications
for each rating and summarize the overall evaluation.
Respond using app domain language; you must not use
technical terminology or mention code details.
Enumerate the problems identified; add an empty
paragraph after each enumeration; no preceding
or following text."""

app_overview = utils.app_overview
user_task = utils.user_task
source_code = utils.source_code

user_prompt = f"""I have a web application written about: {app_overview}
The user’s task in this app view is about: {user_task}.
An image of the app view is provided.
Below is the incomplete code for the app view.
This code includes the view’s user interface.
Source Code:
{source_code}"""


In [12]:
# Example usage
detect_usability_issues("all_courses.png") 

Evaluation of the web application's view on Nielsen's usability principles:

1. **Visibility of System Status (NS01):** 4/5

   The system status is well conveyed by displaying available courses clearly with images, titles, and descriptions. However, there is no loading indicator for cases when the list is being fetched which could enhance status visibility.

2. **User Control and Freedom (NS02):** 3/5

   Users can view courses through clearly marked buttons, but there is no visible option to filter or sort courses, which could provide better control and freedom in navigation.

3. **Help Users Recognize, Diagnose, and Recover from Errors (NS03):** 3/5

   The message "There are no courses yet!" is clear if no courses are available. However, there could be more guidance on refreshing or checking again later to improve error recovery.

4. **Consistency and Standards (NS04):** 4/5

   The design is consistent in terms of layout and button styling. However, the use of different column cla

In [16]:
 # Cognitive Walkthrough
system_prompt_walkthrough = """
You are a UX/UI expert conducting a cognitive walkthrough of a web application to identify usability issues related to 
task completion from a user's perspective."""

demo_user_task = """The user wants to view the profile of the person who uploaded the currently visible video."""

request_for_walkthrough = f"""
You are shown one screenshot of a web interface, before the user takes an action to achieve the task.

Identify usability issues a user may face when attempting the following task:

**User Task**:  
{demo_user_task}

Use the following questions as guidance to help you explore the interface from multiple angles. 
You are NOT limited to one issue per question — you may identify multiple issues inspired by a single question, 
and some issues may relate to more than one question:
- What on the screen might cause confusion to the user when interpreting what they need to do to finish the task?  
- What options are not available on the screen that the user might look for in order to finish the task?  
- In what way does the app not guide the user toward the correct action?  
- What might the user try, think, or misinterpret on the interface?  
- What could be unclear to the user prior to interacting with an element?

### Output Format:
Provide only a **plain enumerated list of issues**, without headings, categories, or extra recommendations.  
Each issue should be written in **1-2 short sentences** and must reflect how a user might **perceive, interpret, or react** to the screen.

### Example Output:

Issues:
1. There are two identical buttons, which could confuse users about their functions and the differences between them.  
2. Nothing is labeled and it might be unclear to the user what button might lead to where. Adding tooltips or labels might help.  
3. The upload icon is placed next to the profile picture, which could lead users to think both do the same thing.
"""

In [23]:
# Walkthrough demos
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")
    
def detect_usability_issues_walkthrough1(image_filename1, system_prompt_walkthrough1, request_for_walkthrough1):
    image_path1 = os.path.join(os.pardir, "images", image_filename1) 
    
    if not os.path.exists(image_path1):
        raise FileNotFoundError(f"Error: The file '{image_path1}' was not found. Check the path.")

    base64_image1 = encode_image(image_path1)

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt_walkthrough1},
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": request_for_walkthrough1},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/png;base64,{base64_image1}",
                        },
                    },
                ],
            },
        ],
    )

    print(response.choices[0].message.content)

In [ ]:
detect_usability_issues_walkthrough1("walkthrough-prev.png", system_prompt_walkthrough, request_for_walkthrough)

Issues:
1. There is no clear label or icon indicating how to view the uploader's profile directly from the video.
2. The uploader’s name, "Davka_overlay," is presented but not highlighted or interactive, which may confuse users looking to access the profile.
3. The uploader’s profile icon is not distinctly visible, possibly leading users to overlook it.
4. The interface lacks a tooltip or indication that clicking the uploader's name or icon would lead to their profile.
5. Users might mistake other visible icons (like the heart or comment icon) for options to view the uploader’s profile.
6. The three-dot menu icon's function is not immediately clear, leading to potential misinterpretation of its purpose.


In [ ]:
# Walkthrough with a simple prompt version for comparison with the previous prompt

system_prompt_walkthrough2 = """
You are a UX/UI expert conducting a cognitive walkthrough of a web application to identify usability issues related to 
task completion from a user's perspective."""

demo_user_task = """The user wants to view the profile of the person who uploaded the currently visible video."""

request_for_walkthrough2 = f"""
You are shown one screenshot of a web interface, before the user takes an action to achieve the task.

Identify usability issues a user may face when attempting the following task:

**User Task**:  
{demo_user_task}

Use the following question as guidance to help you explore the interface from multiple angles. 
You are NOT limited to one issue — you may identify multiple issues inspired by this single question.

Question: What aspects of the interface might confuse the user while attempting to accomplish the task? 
Is complex problem solving needed to figure out what to do and what might the?

### Output Format:
Provide only a **plain enumerated list of issues**, without headings, categories, or extra recommendations.  
Each issue should be written in **1-2 short sentences** and must reflect how a user might **perceive, interpret, or react** to the screen.

### Example Output:

Issues:
1. There are two identical buttons, which could confuse users about their functions and the differences between them.  
2. Nothing is labeled and it might be unclear to the user what button might lead to where. Adding tooltips or labels might help.  
3. The upload icon is placed next to the profile picture, which could lead users to think both do the same thing.
"""

In [28]:
detect_usability_issues_walkthrough1("walkthrough-prev.png", system_prompt_walkthrough2, request_for_walkthrough2)

Issues:
1. The profile icon of the uploader might not be visibly distinct among other interactive elements, leading to confusion.
2. The username is not highlighted or easily identifiable, making it unclear where to click to view the profile.
3. There is no direct label or indication guiding the user to access the uploader's profile.
4. The uploader’s profile picture is small and mixed with other icons, which could cause the user to overlook it.
5. The interface lacks clear affordances or cues to inform users where to click to complete the task.


In [ ]:
# The FINAL prompt with promising results

system_prompt_walkthrough3 = """
You are a UX/UI expert conducting a cognitive walkthrough of a web application to identify usability issues related to 
task completion from a user's perspective."""

demo_user_task = """The user wants to view the profile of the person who uploaded the currently visible video."""
demo_correct_action = """Click the circle icon with the plus symbol above the likes in order to view the user profile."""

request_for_walkthrough3 = f"""
You are shown one screenshot of a web interface, before the user takes an action to achieve his task.

The user task is: 
{demo_user_task}
The expected action to accomplish the given task above is:
{demo_correct_action}

Identify usability issues a user may face when attempting the task.

Use the following question as guidance to help you explore the interface from multiple angles. 
You are NOT limited to one issue per question — you may identify multiple issues inspired by a single question, 
and some issues may relate to more than one question:

- What aspects of the interface might confuse the user when trying to complete the task and figure out the what's the correct action?  
- What is missing or unclear before the user interacts with anything?
- What might the user try instead of the correct action, and why?  

### Output Format:
Provide only a **plain enumerated list of issues** (5 or more issues), without headings, categories, or extra recommendations.  
Each issue should be written in **1-2 short sentences** and must reflect how a user might **perceive, interpret, or react** to the screen.

### Example Output:

Issues:
1. There are two identical buttons, which could confuse users about their functions and the differences between them.  
2. Nothing is labeled and it might be unclear to the user what button might lead to where. Adding tooltips or labels might help.  
3. The upload icon is placed next to the profile picture, which could lead users to think both do the same thing.
"""

In [44]:
detect_usability_issues_walkthrough1("walkthrough-prev.png", system_prompt_walkthrough3, request_for_walkthrough3)

Issues:

1. The circle icon with the plus symbol might be misconstrued as a "follow" button rather than a profile link.
2. There is no label or tooltip indicating that the icon leads to the user profile, causing potential confusion.
3. Users might expect to click the username near video details to view the profile, but that area doesn't appear interactive.
4. The icon's position above the likes could mislead users to think it relates to liking or favoriting instead.
5. The design elements are densely packed, which could make it difficult for users to discern the correct button to click.
6. The presence of a plus symbol may imply adding something rather than viewing a profile, conflicting with user expectations.
7. Users might think the "Profile" option in the sidebar relates to their own profile instead of the video's uploader.


In [ ]:
demo_user_task1 = """The user wants to download a specific video from a list on the right."""
demo_correct_action1 = """Click the three dots next to the video title and click on the download button."""

request_for_walkthrough4 = f"""
You are shown one screenshot of a web interface, before the user takes an action to achieve his task.

The user task is: 
{demo_user_task1}
The expected action to accomplish the given task above is:
{demo_correct_action1}

Identify usability issues a user may face when attempting the task.

Use the following question as guidance to help you explore the interface from multiple angles. 
You are NOT limited to one issue per question — you may identify multiple issues inspired by a single question, 
and some issues may relate to more than one question:

- What aspects of the interface might confuse the user when trying to complete the task and figure out the what's the correct action?  
- What is missing or unclear before the user interacts with anything?
- What might the user try instead of the correct action, and why?  

### Output Format:
Provide only a **plain enumerated list of distinct issues** (5 or more issues), without headings, categories, or extra recommendations.  
Each issue should be written in **1-2 short sentences** and must reflect how a user might **perceive, interpret, or react** to the screen.

### Example Output:

Issues:
1. There are two identical buttons, which could confuse users about their functions and the differences between them.  
2. Nothing is labeled and it might be unclear to the user what button might lead to where. Adding tooltips or labels might help.  
3. The upload icon is placed next to the profile picture, which could lead users to think both do the same thing.
"""

detect_usability_issues_walkthrough1("walkthrough-prev2.png", system_prompt_walkthrough3, request_for_walkthrough4)

Issues:
1. The three dots may not be immediately recognizable as the action menu, causing confusion about where to click.
2. The list of videos is very close together, which might make it difficult to associate the correct set of dots with the video.
3. There is no visual cue or label indicating to users that additional actions can be found under the three dots menu.
4. Users might try clicking the video title itself expecting to be able to download directly, leading to frustration.
5. The placement of the three dots could cause users to accidentally click the wrong video option due to proximity.
6. The download option could be overlooked due to its position in the menu list after opening it.


In [ ]:
system_prompt_walkthrough3 = """
You are a UX/UI expert conducting a cognitive walkthrough of a web application to identify usability issues related to 
task completion from a user's perspective."""

demo_user_task2 = """The user wants to view the profile of the person who uploaded the currently visible video."""
demo_correct_action2 = """Click the circle icon with the plus symbol above the likes in order to view the user profile."""

history = """You are given an image of the web interface **before** the user has taken the correct action. 
Use this interface as a reference for further analyzing the transition and identifing any **potential usability issues**."""

request_for_feedback = f"""
You are shown one screenshot of a web interface, **after** the user has taken the correct action to complete the following task:

**User Task**:  
{demo_user_task2}

**Correct Action Taken by User**:  
{demo_correct_action2}

Your job is to identify usability issues related to how well the interface communicates feedback *after* the action.

### Use the following guiding question:
- After the user does the correct action, will they know that they did the right action and are making progress toward their goal?
- Is complex problem solving needed to interpret the feedback?

### Output Format:
Provide only a **plain enumerated list of issues** (2 or more), without headings, categories, or extra recommendations.  
Each issue should be written in **1-2 short sentences**, based on what a user might **perceive, interpret, or feel** after performing the action.

### Example Output:

Issues:
1. After clicking the icon, nothing visually changes on the screen, leaving the user uncertain if the action worked.  
2. There is no loading indicator or transition effect, so the user might think the interface is unresponsive.  
3. The screen changes but doesn't clearly show that the user is now on the uploader's profile, which might confuse them.
"""


In [55]:
def detect_usability_issues_walkthrough2(image_filename1, image_filename2, system_prompt_walkthrough1, request_for_feedback):
    image_path1 = os.path.join(os.pardir, "images", image_filename1) 
    image_path2 = os.path.join(os.pardir, "images", image_filename2) 
    
    if not os.path.exists(image_path1):
        raise FileNotFoundError(f"Error: The file '{image_path1}' was not found. Check the path.")
    
    if not os.path.exists(image_path2):
        raise FileNotFoundError(f"Error: The file '{image_path2}' was not found. Check the path.")
    
    base64_image1 = encode_image(image_path1)
    base64_image2 = encode_image(image_path2)

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt_walkthrough1},
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": history},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/png;base64,{base64_image1}",
                        },
                    },
                ],
            },
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": request_for_feedback},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/png;base64,{base64_image2}",
                        },
                    },
                ],
            },
        ],
    )

    print(response.choices[0].message.content)

In [56]:
detect_usability_issues_walkthrough2("walkthrough-prev.png", "walkthrough-aft.png", system_prompt_walkthrough3, request_for_feedback)

Issues:
1. The transition to the profile page is not visually distinct, which may leave the user unsure if the action was successful.
2. There is no clear indication or message confirming that the user is now viewing the uploader's profile.
3. The interface lacks a visual cue that the profile view has changed from the previous screen, leading to possible confusion.
4. Important information about the user profile is not highlighted, making it difficult for the user to verify the action.
5. Without a loading indicator, users might suspect a delay or error in loading the profile page.
